## Introducción
La pandemia mundial del COVID-19, desencadenada por el virus SARS-CoV-2, se ha convertido en la crisis sanitaria más grande en la historia reciente, tomando por sorpresa a gobiernos de todo el mundo, paralizando las economías y sus cadenas de suministro, y poniendo a prueba los sistemas de salud pública. En el caso específico de Perú, a pesar de la rápida implementación de medidas de confinamiento, las condiciones estructurales de su economía, marcadas por altos niveles de informalidad, y su sistema de salud, caracterizado por la falta de financiamiento y desarticulación (Ponce de Leon, 2021), han llevado al país a registrar la tasa de mortalidad más alta por COVID-19 a nivel mundial, alcanzando la cifra de 6,552 muertes por millón de habitantes (Reporte de contagios y muertes por COVID-19, Worldometer, 2023).

La expansión del COVID-19 se caracterizó por su magnitud y velocidad, lo que generó la necesidad de una constante actualización de información sobre la evolución de los casos, las medidas gubernamentales de inmovilización, las recomendaciones de seguridad y los avances en el desarrollo de vacunas. Sin embargo, el desconocimiento inicial sobre la naturaleza del virus, su forma de contagio y cómo combatirlo provocó altos niveles de incertidumbre y desinformación, especialmente a través de las redes sociales (Malecki et al., 2021).

En este contexto, el papel de los medios de comunicación oficiales resulta esencial. Por un lado, tienen el potencial de monitorear constantemente la situación y proporcionar informes sobre cualquier desarrollo relevante. Por otro lado, pueden difundir información confiable y relevante, evitando la propagación de información no respaldada. Acceder a información precisa y confiable ha sido fundamental para mitigar la carga de salud mental durante la pandemia (Hoyt et al., 2022). Por lo tanto, es importante que los medios de comunicación brinden una cobertura precisa y equilibrada de la pandemia, permitiendo que las personas tomen decisiones informadas sobre su salud, evitando generar ansiedad y estrés, así como reduciendo la incertidumbre y el pánico en la población.

Ante este panorama, surge la pregunta sobre el tipo de cobertura que los medios de noticias formales tuvieron durante la pandemia, especialmente en países que sufrieron fuertemente sus efectos, como Perú. Para abordar esta interrogante, en este trabajo utilizaremos los tweets de cuatro de los principales medios de noticias nacionales, aplicando técnicas de procesamiento de lenguaje natural, específicamente el topic modelling, con el objetivo de realizar un análisis temático.


## Datos

Las fuentes de datos utilizadas en este análisis son los tweets publicados por las cuentas oficiales de cuatro de los principales medios de noticias peruanos: El Comercio, La República, Willax y Ojo Público. La base de datos cuenta con un total de 931460 tweets, que abarcan un periodo que va desde el año 2016 hasta el año 2023, lo cual nos permitirá analizar la evolución de la cobertura de noticias relacionadas a la pandemia del COVID-19.


## Pasos de análisis

- Filtrado: se seleccionaran los tuits que estén relacionados con el tema del Covid-19. Las palabras elegidas para la filtración son las siguientes ['covid', 'coronavirus', 'covid-19', 'pandemia', 'cuarentena']

- Limpieza: se tokenizaran los tuits (transformando el texto a minúscilas, eliminando acentos, stompwords,etc.) para poder trabajar con un texto limpio.
- Una vez filtrados y limpiados los tweets se eliminaran palabras del tópico general (coronavirus, covid-19, covid) ,para descubrir subtemas o aspectos más específicos en los tuits y evitar que los topicos identificados contengan estas palabras mas generales.

- Topic modelling: se aplicaran las técnicas de Non Negative Matrix Factorization (NNMF) y Latent Dirichlet Allocation (LDA) teniendo como parámetro un número de 10 componentes (tópicos)

- Comparar: identificaremos cual modelo permite identificar de manera mas clara los topicos en el corpus


## Preparación del texto

Importamos paquetes a utilizar

In [8]:
import pandas as pd
import numpy as np
import os
import csv
import datetime
import json
import re
#from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.manifold import TSNE
from nltk.corpus import stopwords # nltk = package for human language data processing.
import os
import csv
import datetime
import json
import re
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mauro\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

Seteamos el directorio

In [6]:
directorio = "H:\Mi unidad\PLN\DatosParaTrabajos\Medios"

files = os.listdir(directorio)
files

['Copy of elcomercio_peru.json',
 'Copy of larepublica_pe.json',
 'Copy of willaxtv.json',
 'Copy of ojo_publico.json']

Guardamos los tuits en un data frame

In [9]:
data=pd.DataFrame()
t=0
for i in files:
  path= directorio+'/'+ i
  with open(path, 'r', encoding="utf-8") as f:
    pf = json.load(f)
    df_aux=pd.DataFrame(pf)
    df_aux=df_aux[:-1]
    df_aux['hand']=i[:-5]
    data = pd.concat([data, df_aux], ignore_index=True, axis=0).reset_index(drop=True)
    t+=1

data.head()

,id,text,created_at,author,type,metrics,texto_respuesta,inicio_politician,hand
0,"""1623832899702095873""","""BCR mantuvo la tasa de inter\u00e9s de refere...","""2023-02-09T23:55:07.000Z""","""233033221""","""original""","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,NaN,Copy of elcomercio_peru
1,"""1623829559308156928""","""Centro de Lima: integrantes de la CGTP se mov...","""2023-02-09T23:41:50.000Z""","""233033221""","""original""","{'retweet_count': 4, 'reply_count': 5, 'like_c...",NaN,NaN,Copy of elcomercio_peru
2,"""1623828283463766018""","""Los crueles ataques racistas de mujer contra ...","""2023-02-09T23:36:46.000Z""","""233033221""","""original""","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,NaN,Copy of elcomercio_peru
3,"""1623825571770781697""","""Asesinato en San Miguel: el paso a paso de c\...","""2023-02-09T23:26:00.000Z""","""233033221""","""original""","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,NaN,Copy of elcomercio_peru
4,"""1623824292130525185""","""EE.UU. anuncia 85 millones de d\u00f3lares en...","""2023-02-09T23:20:55.000Z""","""233033221""","""original""","{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,NaN,Copy of elcomercio_peru


In [10]:
data.shape

(931460, 9)

La base de datos original cuenta con 931 460 tweets


### Filtrado por temática


In [11]:
#Limpiamos tweets preliminarmente

def clean_accents(tweet):
    tweet = re.sub(r"[àáâãäå]", "a", tweet)
    tweet = re.sub(r"ç", "c", tweet)
    tweet = re.sub(r"[èéêë]", "e", tweet)
    tweet = re.sub(r"[ìíîï]", "i", tweet)
    tweet = re.sub(r"[òóôõö]", "o", tweet)
    tweet = re.sub(r"[ùúûü]", "u", tweet)
    tweet = re.sub(r"[ýÿ]", "y", tweet)

    return tweet


In [12]:
f1 = lambda x: re.sub('[,\.!?:/@"]', ' ', x) #elimina caracteres especiales
f2 = lambda x: str(x).lower() # poner en minúsula


In [13]:
data['filtrado'] =data['text'].apply(f1).apply(f2).apply(clean_accents)


Creamos lista con palabras vinculadas a la pandemia por el covid-19 y una nueva base de datos que incluya solo los tweets filtrados por tema

In [14]:
palabras = ['covid', 'covid19' ,'coronavirus', 'covid-19', 'pandemia', 'cuarentena']
data1 = data[data['filtrado'].str.contains('|'.join(palabras))]


In [15]:
data1.head()

,id,text,created_at,author,type,metrics,texto_respuesta,inicio_politician,hand,filtrado
64,"""1623740774348165121""","""Carnaval Huaracino: regresa festividad tras d...","""2023-02-09T17:49:02.000Z""","""233033221""","""original""","{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,NaN,Copy of elcomercio_peru,carnaval huaracino regresa festividad tras d...
90,"""1623678417517400067""","""Minsa reporta 7 decesos y 143 nuevos contagio...","""2023-02-09T13:41:15.000Z""","""233033221""","""original""","{'retweet_count': 1, 'reply_count': 1, 'like_c...",NaN,NaN,Copy of elcomercio_peru,minsa reporta 7 decesos y 143 nuevos contagio...
107,"""1623549976310685696""","""China indica un descenso de muertes e ingreso...","""2023-02-09T05:10:53.000Z""","""233033221""","""original""","{'retweet_count': 1, 'reply_count': 0, 'like_c...",NaN,NaN,Copy of elcomercio_peru,china indica un descenso de muertes e ingreso...
202,"""1623356768330911749""","""\u201cLa pandemia del Covid-19 introdujo much...","""2023-02-08T16:23:08.000Z""","""233033221""","""replied_to""","{'retweet_count': 0, 'reply_count': 1, 'like_c...","""Al ser consultado sobre los motivos por los q...",NaN,Copy of elcomercio_peru,\u201cla pandemia del covid-19 introdujo much...
253,"""1623308835292061697""","""Minsa reporta 2 decesos y 177 nuevos contagio...","""2023-02-08T13:12:40.000Z""","""233033221""","""original""","{'retweet_count': 1, 'reply_count': 0, 'like_c...",NaN,NaN,Copy of elcomercio_peru,minsa reporta 2 decesos y 177 nuevos contagio...


In [16]:
data1.shape

(69540, 10)

La nueva base de datos cuenta con 69 540 tweets

### Limpieza del texto

Creamos las siguientes funciones:
- clean_accents: elimina los acentos
- f1 = eliminar caracteres especiales
- f2 = convertir el texto a minúscula
- f3 = eliminar stop words
- f4 = eliminar palabras con menos de 3 de largo


In [17]:
def clean_accents(tweet):
    tweet = re.sub(r"[àáâãäå]", "a", tweet)
    tweet = re.sub(r"ç", "c", tweet)
    tweet = re.sub(r"[èéêë]", "e", tweet)
    tweet = re.sub(r"[ìíîï]", "i", tweet)
    tweet = re.sub(r"[òóôõö]", "o", tweet)
    tweet = re.sub(r"[ùúûü]", "u", tweet)
    tweet = re.sub(r"[ýÿ]", "y", tweet)

    return tweet

In [18]:
f1 = lambda x: re.sub('[,\.!?:/@"]', ' ', x)
f2 = lambda x: str(x).lower()
stop= stopwords.words("spanish")

In [19]:
print(stop)

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 'e

In [20]:
f3 = lambda x: ' '.join(w.encode('latin-1', errors='ignore').decode('latin-1') for w in x.split() if not w in stop)
f4 = lambda x: ' '.join(w.encode('latin-1', errors='ignore').decode('latin-1') for w in x.split() if len(w) > 3)

Reemplazamos el salto de línea por un espacio

In [21]:
data1['text_clean'] = data1['text'].str.encode('latin-1', errors='ignore').str.decode('unicode_escape')

data1['text_clean'] = data1['text_clean'].str.replace('\n', ' ')

data1['text_clean'] = data1['text_clean'].str.replace('\(', " ", regex=True).str.replace('\)', " ", regex=True)

data1['text_clean'] = data1['text_clean'].str.replace('[\)\[\]‐#@]', " ", regex=True)

data1['text_clean'] =data1['text_clean'].apply(f1).apply(f2).apply(f3).apply(f4).apply(clean_accents)

data1.head()


C:\Users\mauro\AppData\Local\Temp\ipykernel_22632\767568336.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['text_clean'] = data1['text'].str.encode('latin-1', errors='ignore').str.decode('unicode_escape')
C:\Users\mauro\AppData\Local\Temp\ipykernel_22632\767568336.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['text_clean'] = data1['text_clean'].str.replace('\n', ' ')
C:\Users\mauro\AppData\Local\Temp\ipykernel_22632\767568336.py:5: SettingWithCopyWarning: 
A value is trying to be set 

,id,text,created_at,author,type,metrics,texto_respuesta,inicio_politician,hand,filtrado,text_clean
64,"""1623740774348165121""","""Carnaval Huaracino: regresa festividad tras d...","""2023-02-09T17:49:02.000Z""","""233033221""","""original""","{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,NaN,Copy of elcomercio_peru,carnaval huaracino regresa festividad tras d...,carnaval huaracino regresa festividad tras año...
90,"""1623678417517400067""","""Minsa reporta 7 decesos y 143 nuevos contagio...","""2023-02-09T13:41:15.000Z""","""233033221""","""original""","{'retweet_count': 1, 'reply_count': 1, 'like_c...",NaN,NaN,Copy of elcomercio_peru,minsa reporta 7 decesos y 143 nuevos contagio...,minsa reporta decesos nuevos contagios covid-1...
107,"""1623549976310685696""","""China indica un descenso de muertes e ingreso...","""2023-02-09T05:10:53.000Z""","""233033221""","""original""","{'retweet_count': 1, 'reply_count': 0, 'like_c...",NaN,NaN,Copy of elcomercio_peru,china indica un descenso de muertes e ingreso...,china indica descenso muertes ingresos hospita...
202,"""1623356768330911749""","""\u201cLa pandemia del Covid-19 introdujo much...","""2023-02-08T16:23:08.000Z""","""233033221""","""replied_to""","{'retweet_count': 0, 'reply_count': 1, 'like_c...","""Al ser consultado sobre los motivos por los q...",NaN,Copy of elcomercio_peru,\u201cla pandemia del covid-19 introdujo much...,pandemia covid-19 introdujo digitalizacion pub...
253,"""1623308835292061697""","""Minsa reporta 2 decesos y 177 nuevos contagio...","""2023-02-08T13:12:40.000Z""","""233033221""","""original""","{'retweet_count': 1, 'reply_count': 0, 'like_c...",NaN,NaN,Copy of elcomercio_peru,minsa reporta 2 decesos y 177 nuevos contagio...,minsa reporta decesos nuevos contagios covid-1...


Verificamos los cambios en el texto 

In [23]:
data1[['text','text_clean']].head(5)

,text,text_clean
64,"""Carnaval Huaracino: regresa festividad tras d...",carnaval huaracino regresa festividad tras año...
90,"""Minsa reporta 7 decesos y 143 nuevos contagio...",minsa reporta decesos nuevos contagios covid-1...
107,"""China indica un descenso de muertes e ingreso...",china indica descenso muertes ingresos hospita...
202,"""\u201cLa pandemia del Covid-19 introdujo much...",pandemia covid-19 introdujo digitalizacion pub...
253,"""Minsa reporta 2 decesos y 177 nuevos contagio...",minsa reporta decesos nuevos contagios covid-1...


In [24]:
pborrar = ['https', 'covid', 'covid19', 'coronavirus', 'covid-19', 'rt', 't', 'como' ,'co','tras' ,'htp','https','fdfq7gkriy','noticias', 'ojoaldato','loultimo', '4ajelmloq2', 'bestcable', 'vivo', 'c1191', 'sigue', 'directv', 'movistartv', 'clarotv', '52rqfuppqu', 'vivo', 'como', 'suscribete', 'milagrosleivaentrevista', 'video', 'noticia', 'suscriptores', 'contenido', '19','covid' 'comercio', 'solo', 'aqui']
print(pborrar)

['https', 'covid', 'covid19', 'coronavirus', 'covid-19', 'rt', 't', 'como', 'co', 'tras', 'htp', 'https', 'fdfq7gkriy', 'noticias', 'ojoaldato', 'loultimo', '4ajelmloq2', 'bestcable', 'vivo', 'c1191', 'sigue', 'directv', 'movistartv', 'clarotv', '52rqfuppqu', 'vivo', 'como', 'suscribete', 'milagrosleivaentrevista', 'video', 'noticia', 'suscriptores', 'contenido', '19', 'covidcomercio', 'solo', 'aqui']


In [25]:
f5 = lambda x: ' '.join(w.encode('latin-1', errors='ignore').decode('latin-1') for w in x.split() if not w in pborrar)

In [26]:
data1['text_clean'] =data1['text_clean'].apply(f5)

data1[['text','text_clean']]


C:\Users\mauro\AppData\Local\Temp\ipykernel_22632\239499121.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['text_clean'] =data1['text_clean'].apply(f5)


,text,text_clean
64,"""Carnaval Huaracino: regresa festividad tras d...",carnaval huaracino regresa festividad años con...
90,"""Minsa reporta 7 decesos y 143 nuevos contagio...",minsa reporta decesos nuevos contagios ultimas...
107,"""China indica un descenso de muertes e ingreso...",china indica descenso muertes ingresos hospita...
202,"""\u201cLa pandemia del Covid-19 introdujo much...",pandemia introdujo digitalizacion publicidad p...
253,"""Minsa reporta 2 decesos y 177 nuevos contagio...",minsa reporta decesos nuevos contagios ultimas...
...,...,...
904195,"""Mientras el coronavirus y el p\u00e1nico se e...",mientras panico extienden mundo dengue represe...
904196,"""Mientras la discusi\u00f3n se centra en el #c...",mientras discusion centra pocos hablan dengue ...
904199,"""#Fotoreportaje Mientras el coronavirus se ext...",fotoreportaje mientras extienden varios paises...
922505,"""Crisis humanitaria en Venezuela pone en pelig...",crisis humanitaria venezuela pone peligro esfu...



## Método 1: Non Negative Matrix Factorization - NNMF

Para identificar los tópicos necesitamos reducir la dimensionalidad del texto, es decir, descomponer nuestra matriz ${D}$ en dos matrices:
- ${Q}$  = tweets x tópicos
- ${P}$  = topicos x terminos




$\underbrace{D}_{n\times d} \approx \underbrace{Q}_{n \times k} \overbrace{\Sigma}^{k\times k} \underbrace{P^{T}}_{k \times d}$

El método NNMF construye las matrices utilizando la frecuencia de términos (TF, Term Frequency)



### Matriz de término frecuencia (TF)

In [27]:
tfidf = TfidfVectorizer()

La matriz se construye a partir de los tweets limpios

In [28]:
tfidf = TfidfVectorizer()

tfs = tfidf.fit_transform(data1['text_clean'])

# Tuits x términos

print(tfs.toarray().shape)


MemoryError: Unable to allocate 51.4 GiB for an array with shape (69540, 99218) and data type float64

In [ ]:
tfs.toarray()

Creamos un diccionario de terminos

In [ ]:
print("Nuestro corpus tiene {} terminos:".format(len(tfidf.vocabulary_)))

In [ ]:
tfidf.vocabulary_

In [ ]:
words = np.array(tfidf.get_feature_names_out())
print(len(words))


Tenemos 99218 terminos en total

### Parámetros

Probaremos diferentes números de componentes o tópicos ${k}$ para ver cual permite identificar mejor topics diferentes.

Guardamos la clase NMF como objeto y le damos parámetros iniciales

In [ ]:
nmf = NMF(n_components=10, solver='mu', init='nndsvda')

Q = nmf.fit_transform(tfs)
P = nmf.components_

In [ ]:
Q.shape

69540 tweets x 10 tópicos

In [ ]:
P.shape

10 topicos x 99220 terminos


## Palabras más probables por tópico


In [ ]:
for i, topic in enumerate(P):
     print("Topic {}: {}".format(i + 1, ",".join([str(x) for x in words[topic.argsort()[-10:]]])))


Con un parámetro de 10 componentes se identifican palabras  que cubren una variedad de aspectos relacionados con COVID-19, como informes de casos y fallecimientos, vacunación, situación internacional, medidas gubernamentales. Sin embargo, las palabras de algunos temas se solapan, como en el caso del Topic 1 y el Topic 5, pues ambos incluyen "fallecidos", "decesos", "contagios" y "minsa_peru". Esto sugiere que estos dos temas pueden estar relacionados con la situación de la salud y los informes de casos y fallecimientos. Para ello probaremos reducir el numero de topics en el modelo y compararemos los resultados.

**7 topics**

In [ ]:
nmf = NMF(n_components=7, solver='mu', init='nndsvda')

words = np.array(tfidf.get_feature_names_out())

Q = nmf.fit_transform(tfs)
P = nmf.components_
most_likely_topic = Q.argmax(axis=1)
for i, topic in enumerate(P):
     print("Topic {}: {}".format(i + 1, ",".join([str(x) for x in words[topic.argsort()[-10:]]])))


-  Topic 1:  relacionado con los informes de fallecimientos y contagios proporcionados por el Ministerio de Salud (Minsa).
- Topic 2: se centra en la vacunación en Perú, específicamente en la tercera dosis de pfizer, dirigida a personas mayores.
- Topic 3: está relacionado con los casos, contagios y muertes en Argentina, Brasil y México
- Topic 4: aborda las vacunas en Brasil (el país funciono como laboratorio de pruebas y realizó campaña de vacunacion masiva)
- Topic 5: está relacionado con los registros y eventos en diferentes regiones del país, como el aumento de casos y fallecimientos.
- Topic 6: relacionado con informes del minsa sobre casos y pacientes recuperados
 / dados de alta.
- Topic 7: relacionado con el primer caso, martin vizcarra y medidas gubernamentales, como la cuarentena.


In [ ]:
Q.shape
most_likely_topic = Q.argmax(axis=1)

Creamos una columna en el data frame que asigne el topic más probable a cada tweet

In [ ]:
data1['most_likely_topic'] = most_likely_topic

## Análisis descriptivo

Contamos la frecuencia y proporción de cada tópico


In [ ]:
topic_counts = data1['most_likely_topic'].value_counts()
topic_counts

In [ ]:
topic_proportions = topic_counts / len(data1)
print(topic_proportions)

Los 5 temas más comunes son:
- Topic 7: Anuncios de Martin Vizcarra  (33 708 tweets / 48%)
- Topic 2: Vacunacion en el Perú  (11 052 tweets / 16% )
- Topic 3: Emergencia sanitaria mundial en el año 2021  (13 555 tweets / 19%)
- Topic 8: Proceso de vacunación (6293 tweets / 9%)
- Topic 2: Contagios y casos intenacionales (4404 tweets / 6%)


In [ ]:
topic_percentages = topic_proportions* 100

In [ ]:
# Histograma de barras
plt.figure(figsize=(10, 6))
topic_percentages.plot(kind='bar')
plt.xlabel('Temas')
plt.ylabel('Porcentaje')
plt.title('Porcentaje de tweets por temas')
plt.show()

Se aprecia claramente que el topic 7 predomina en relación a los tuits que hablan sobre la pandemia. Esto puede deberse a que el primer caso de COVID-19 fue un evento altamente significativo, por lo que recibió una gran cobertura mediática. Además, como presidente en ese momento, Martin Vizcarra tenía un papel clave en la toma de decisiones y la implementación de medidas para contener la propagación del virus, siendo un tema de elevada difusión mediática.

## Visualización: scatter plot

Calculamos la proporción de cada tópico para

In [ ]:
topic_proportions2 = Q.sum(axis=0) / Q.sum()
topic_proportions2

Q.sum(axis=0)

Q.sum()

Factoriza matriz en dos dimensiones

In [ ]:
# Apply t-SNE to reduce the dimensionality of the data
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(Q)

Creamos lista de top 10 palabras más importantes por tópico y scatter plot

In [ ]:
top_words = []
for topic in P:
    word_idx = topic.argsort()[-10:]
    top_words.append([words[idx] for idx in word_idx])

In [ ]:
# Create scatter plot with topic colors and labels a cada documento le asignamos color segun  código
plt.figure(figsize=(10, 10))
sns.scatterplot(
    x=tsne_results[:,0],
    y=tsne_results[:,1],
    hue=Q.argmax(axis=1), # Color-code points by their assigned topic
    palette=sns.color_palette("hls", nmf.n_components),
    legend="full",
    alpha=0.7
)

In [ ]:
# Add topic labels to plot
for i, words in enumerate(top_words):
    label = "Topic {}: {}".format(i+1, ", ".join(words))
    x_pos = tsne_results[Q.argmax(axis=1) == i, 0].mean()
    y_pos = tsne_results[Q.argmax(axis=1) == i, 1].mean()
    plt.text(x_pos, y_pos, label, fontsize=10, bbox=dict(facecolor='white', alpha=0.8))

plt.title("NMF: Topicos sobre el Covid-19 en los medios de comunicación peruanos({} Topics)".format(nmf.n_components))
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")
plt.show()

## Método 2: Latent Dirichlet Allocation - LDA

Creamos lista con los tweets

In [ ]:
data2 = data1.text_clean.values.tolist()

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

data_words = list(sent_to_words(data2))

id2word = corpora.Dictionary(data_words)

print(id2word[4])
print(len(id2word))

In [ ]:
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
from pprint import pprint
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10,
                                       passes=10)

pprint(lda_model.print_topics(num_words=10))

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
lda_tweets = LDA(n_components=10, random_state=42)
lda_tweets.fit(tfs)

In [ ]:
for idx, topic in enumerate(lda_tweets.components_):
    print(f"las 10 palabras más importantes por tema #{idx}:")
    print([tfidf.get_feature_names_out()[i] for i in topic.argsort()[-10:]])
    print('')


Podemos apreciar que el modelo LDA no está generando temas tan diferenciados, al observar una elevada repetición de las mismas palabras en los 10 temas. Esto se debe principalmente a que, a diferencia del NNMF, el LDA no utiliza términos frecuencia, sino que también toma en cuenta la frecuencia en relación a la distribucion de la palabra en el total del corpus. Probaremos cambiando los parámetros para evaluar si hay una mejora

**7 topics**

In [ ]:
from pprint import pprint
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=7,
                                       passes=10)

pprint(lda_model.print_topics(num_words=10))

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
lda_tweets = LDA(n_components=7, random_state=42)
lda_tweets.fit(tfs)

In [ ]:
for idx, topic in enumerate(lda_tweets.components_):
    print(f"las 10 palabras más importantes por tema #{idx}:")
    print([tfidf.get_feature_names_out()[i] for i in topic.argsort()[-10:]])
    print('')

El reducir el número de topics no parece ayudar en la identificación de temas claramente distingibles entre si:
cuaretena aparece en los tópicos 0, 1, 4 y 5.
vacuna: Aparece en los tópicos 0, 1, 3, 4 y 5.
casos: Aparece en los tópicos 1, 2, 3, 4, 5 y 6.
pandemia: Aparece en los tópicos 0, 1, 2, 3, 4 y 5.
salud: Aparece en los tópicos 1, 2, 5 y 6.
perú: Aparece en los tópicos 0, 1, 2, 3, 4, 5 y 6.
muertes: Aparece en los tópicos 0, 2, 3 y 4.
millones: Aparece en los tópicos 0, 1, 2 y 5.


## Conclusiones

El método de Non Negative Matrix Factorization fue el más efectivo para aproximarnos a los principales temas los tuits de los medios de noticias sobre el Covid-19. En contraste, el método LDA tuvo problemas para generar temas diferenciados, identificando palabras más generales que se repetian en los distintos tópicos.

El NNMF permite aprecia que los medios de noticias realizaron un reporte continuo sobre el covid reportando las cifras actualizadas de contagios y fallecidos a nivel nacional proporcionadas por el MINSA y las cifras de distintos países a nivel internacional. El tema predominante fueron las medidas tomadas por el gobierno de Vizcarra. Finalmente, otro tema central de información fueron los procesos de vacunación, que contribuyeron con acelerar su proceso y que se lleve a cabo de manera exitosa.




## Bibliografía

- Hoyt, D. L., Hiserodt, M., Gold, A. K., Milligan, M. A., & Otto, M. W. (2022). Is Ignorance Bliss? Examining the Effect of News Media Exposure on Anxiety and Depression During the COVID-19 Pandemic. The Journal of nervous and mental disease, 210(2), 91–97. https://doi.org/10.1097/NMD.0000000000001434
- Malecki, K. M., Keating, J. A., & Safdar, N. (2021). Crisis communication and public perception of COVID-19 risk in the era of social media. Clinical infectious diseases, 72(4), 697-702.
- Ponce de León, Z. (2021). Sistema de Salud en el Perú y el COVID-19.
- Worldometer (2023) Reporte de contagios y muertes por COVID 19.
Recuperado el 7 de julio de 2023 de https://www.worldometers.info/coronavirus/
